In [401]:
import pandas as pd
import numpy as np
import uuid
import os
import jsonlines
import datetime
import uuid

In [402]:
all_sessions_raw = pd.read_json("../output/raw/sessions_all_crawlies.json")

In [403]:
all_eposters_raw = pd.read_json("../output/raw/e_posters_all_crawlies.json")

In [404]:
all_sessions_raw.iloc[0]

id                            8e9849b4-69ba-45b1-bb80-3f6cba5503a2
class                                                      session
range                                                          8.0
order                                                          3.0
event_id                      95271c09-7598-40e8-91b5-61712e726e86
title            Pfizer Oncology - Treatment considerations in ...
session_type                          Industry Satellite Symposium
content_id                                                     NaN
role                                                           NaN
full_name                                                      NaN
first_name                                                     NaN
last_name                                                      NaN
second_name                                                    NaN
country                                                        NaN
city                                                          

## All sessions

In [405]:
# filter sessions from all session data (sessions, pres, persons)
sessionsdf = all_sessions_raw[all_sessions_raw["class"] == "session"][[
        "id", "range", "order", "event_id", "title", "session_type", "location", "start_time", "end_time"
    ]]

In [406]:
sessionsdf.head()

,id,range,order,event_id,title,session_type,location,start_time,end_time
0,8e9849b4-69ba-45b1-bb80-3f6cba5503a2,8.0,3.0,95271c09-7598-40e8-91b5-61712e726e86,Pfizer Oncology - Treatment considerations in ...,Industry Satellite Symposium,NaN,NaT,NaT
11,e995dd2b-68a4-409c-9139-35741e64296b,1.0,11.0,95271c09-7598-40e8-91b5-61712e726e86,EONS13: Supporting informal carers,Channel 4,Channel 4,2020-09-20 16:20:00,2020-09-20 17:50:00
23,ca8ab007-e0a3-48fe-b774-bef524571aa9,8.0,11.0,95271c09-7598-40e8-91b5-61712e726e86,Takeda - Oncogenic drivers in advanced NSCLC: ...,Industry Satellite Symposium,NaN,NaT,NaT
35,bacee142-0f47-4d12-b9c1-d87756d0d0ce,8.0,10.0,95271c09-7598-40e8-91b5-61712e726e86,Pfizer Oncology - The complete picture: Managi...,Industry Satellite Symposium,NaN,NaT,NaT
51,e0318b1e-1be7-4ba4-8fbd-229928c3eaaa,8.0,8.0,95271c09-7598-40e8-91b5-61712e726e86,MSD - Evolving treatment paradigms for head an...,Industry Satellite Symposium,NaN,NaT,NaT


In [407]:
# convert range and order to integers
sessionsdf["range"] = sessionsdf["range"].apply(lambda x: int(x))
sessionsdf["order"] = sessionsdf["order"].apply(lambda x: int(x))

In [408]:
# visualize duplicated sessions
sessionsdf[sessionsdf.duplicated(['title'])]

,id,range,order,event_id,title,session_type,location,start_time,end_time
4980,f3bc7a75-21eb-4685-9da4-2626cd0b811c,1,2,95271c09-7598-40e8-91b5-61712e726e86,ESMO Colloquium supported by Eisai: Controvers...,Channel 1,Channel 1,2020-09-15 13:00:00,2020-09-15 14:30:00
4998,990c248e-ce91-49f0-9312-ee1fb04f6bc3,1,1,95271c09-7598-40e8-91b5-61712e726e86,ESMO Colloquium supported by Eli Lilly and Com...,Channel 1,Channel 1,2020-09-23 13:00:00,2020-09-23 14:30:00
5016,b321dd63-ad57-48c6-974c-9b928032a3b5,1,0,95271c09-7598-40e8-91b5-61712e726e86,ESMO Colloquium supported by Eli Lilly and Com...,Channel 1,Channel 1,2020-09-22 13:00:00,2020-09-22 14:30:00
5127,440866c5-b3a6-4ce8-990e-ae991bd3406e,20,2,95271c09-7598-40e8-91b5-61712e726e86,ESMO Colloquium supported by Eli Lilly and Com...,ESMO Colloquium,NaN,NaT,NaT
5158,ab8b5d8b-84cd-4818-b897-517dd985a5dd,20,1,95271c09-7598-40e8-91b5-61712e726e86,ESMO Colloquium supported by Eli Lilly and Com...,ESMO Colloquium,NaN,NaT,NaT
5228,449e2c36-fe0e-49dd-8df8-85e8c176b90f,20,0,95271c09-7598-40e8-91b5-61712e726e86,ESMO Colloquium supported by Eisai: Controvers...,ESMO Colloquium,NaN,NaT,NaT


In [409]:
len(sessionsdf)

211

In [410]:
# visualize unique titles
unique_titles = sessionsdf[sessionsdf.duplicated(['title'])]["title"].unique()

In [411]:
unique_titles

array(['ESMO Colloquium supported by Eisai: Controversies endometrial cancer: Update on ESMO clinical practice guidelines (CPG) and future treatment direction',
       'ESMO Colloquium supported by Eli Lilly and Company: Are single agent TKIs still Standard of Care in EGFR mutant Non-Small Cell Lung Cancer or should they be combined with other agents?',
       'ESMO Colloquium supported by Eli Lilly and Company: Precision Medicine Implementation in the Clinic: RET as a potential target in cancer treatment'],
      dtype=object)

In [412]:
ids_mapping = []

In [413]:
# group ids corresponding to duplicated sessions and map same id for all 
for title in unique_titles:
    print(title)
    ids = sessionsdf[sessionsdf["title"]==title]["id"].unique()
    print(list(ids))
    ids_mapping.append(list(ids))
    sessionsdf["id"] = sessionsdf["id"].apply(
            lambda x: ids[0] if x in ids else x)

ESMO Colloquium supported by Eisai: Controversies endometrial cancer: Update on ESMO clinical practice guidelines (CPG) and future treatment direction
['4fe12b19-280a-4286-a84a-979e04263a29', 'f3bc7a75-21eb-4685-9da4-2626cd0b811c', '449e2c36-fe0e-49dd-8df8-85e8c176b90f']
ESMO Colloquium supported by Eli Lilly and Company: Are single agent TKIs still Standard of Care in EGFR mutant Non-Small Cell Lung Cancer or should they be combined with other agents?
['4b030629-e531-4302-90ca-059b452b8807', '990c248e-ce91-49f0-9312-ee1fb04f6bc3', '440866c5-b3a6-4ce8-990e-ae991bd3406e']
ESMO Colloquium supported by Eli Lilly and Company: Precision Medicine Implementation in the Clinic: RET as a potential target in cancer treatment
['8931e87d-dcba-49e9-b62f-0c293a479271', 'b321dd63-ad57-48c6-974c-9b928032a3b5', 'ab8b5d8b-84cd-4818-b897-517dd985a5dd']


In [414]:
sessionsdf["id"].unique()

array(['8e9849b4-69ba-45b1-bb80-3f6cba5503a2',
       'e995dd2b-68a4-409c-9139-35741e64296b',
       'ca8ab007-e0a3-48fe-b774-bef524571aa9',
       'bacee142-0f47-4d12-b9c1-d87756d0d0ce',
       'e0318b1e-1be7-4ba4-8fbd-229928c3eaaa',
       '82fd33f8-d528-4699-8bf3-8a8ad24b5e46',
       'ece7729b-77ac-43a5-abd3-d34e4b1cb6cc',
       '7c1c6cb3-d2ba-4938-8ed2-914f7d503245',
       'ecc14d1d-5d40-4dbb-9d1d-0120aa493e92',
       '327bfcbe-df29-4a43-82aa-cbec96f7019c',
       '62465bc8-3733-40a6-bc49-d8a30ecc8c39',
       '5422da19-8361-46ff-91f9-60a9862e5f30',
       '8a5557cb-1f7e-46b2-a568-759fe83cb26d',
       'e165ef71-0437-49aa-9efb-bd753117a97a',
       'e361c5e9-b20d-4b3e-870f-f297438092fd',
       '458d8bd1-a7c7-4572-84e5-c14023af247f',
       'a47c7549-e5d8-4a2b-9415-5c1b14f07fd4',
       '576f4ced-fbd5-4ab1-b5c6-3cf35becca08',
       'ea8d1987-d612-4398-b84c-fc7d707dc893',
       'b90481d0-96c4-49c8-8674-a4b61c057f5b',
       '91811043-0728-4318-9926-04d8762d9aaa',
       'fcdc7

In [415]:
sessionsdf[sessionsdf.duplicated(['title'])]

,id,range,order,event_id,title,session_type,location,start_time,end_time
4980,4fe12b19-280a-4286-a84a-979e04263a29,1,2,95271c09-7598-40e8-91b5-61712e726e86,ESMO Colloquium supported by Eisai: Controvers...,Channel 1,Channel 1,2020-09-15 13:00:00,2020-09-15 14:30:00
4998,4b030629-e531-4302-90ca-059b452b8807,1,1,95271c09-7598-40e8-91b5-61712e726e86,ESMO Colloquium supported by Eli Lilly and Com...,Channel 1,Channel 1,2020-09-23 13:00:00,2020-09-23 14:30:00
5016,8931e87d-dcba-49e9-b62f-0c293a479271,1,0,95271c09-7598-40e8-91b5-61712e726e86,ESMO Colloquium supported by Eli Lilly and Com...,Channel 1,Channel 1,2020-09-22 13:00:00,2020-09-22 14:30:00
5127,4b030629-e531-4302-90ca-059b452b8807,20,2,95271c09-7598-40e8-91b5-61712e726e86,ESMO Colloquium supported by Eli Lilly and Com...,ESMO Colloquium,NaN,NaT,NaT
5158,8931e87d-dcba-49e9-b62f-0c293a479271,20,1,95271c09-7598-40e8-91b5-61712e726e86,ESMO Colloquium supported by Eli Lilly and Com...,ESMO Colloquium,NaN,NaT,NaT
5228,4fe12b19-280a-4286-a84a-979e04263a29,20,0,95271c09-7598-40e8-91b5-61712e726e86,ESMO Colloquium supported by Eisai: Controvers...,ESMO Colloquium,NaN,NaT,NaT


In [416]:
# drop duplicated sessions
sessionsdf = sessionsdf.drop_duplicates(subset="title", keep='first')

In [417]:
sessionsdf[sessionsdf.duplicated(['title'])]

,id,range,order,event_id,title,session_type,location,start_time,end_time


In [418]:
sessionsdf.head()

,id,range,order,event_id,title,session_type,location,start_time,end_time
0,8e9849b4-69ba-45b1-bb80-3f6cba5503a2,8,3,95271c09-7598-40e8-91b5-61712e726e86,Pfizer Oncology - Treatment considerations in ...,Industry Satellite Symposium,NaN,NaT,NaT
11,e995dd2b-68a4-409c-9139-35741e64296b,1,11,95271c09-7598-40e8-91b5-61712e726e86,EONS13: Supporting informal carers,Channel 4,Channel 4,2020-09-20 16:20:00,2020-09-20 17:50:00
23,ca8ab007-e0a3-48fe-b774-bef524571aa9,8,11,95271c09-7598-40e8-91b5-61712e726e86,Takeda - Oncogenic drivers in advanced NSCLC: ...,Industry Satellite Symposium,NaN,NaT,NaT
35,bacee142-0f47-4d12-b9c1-d87756d0d0ce,8,10,95271c09-7598-40e8-91b5-61712e726e86,Pfizer Oncology - The complete picture: Managi...,Industry Satellite Symposium,NaN,NaT,NaT
51,e0318b1e-1be7-4ba4-8fbd-229928c3eaaa,8,8,95271c09-7598-40e8-91b5-61712e726e86,MSD - Evolving treatment paradigms for head an...,Industry Satellite Symposium,NaN,NaT,NaT


In [419]:
ids_mapping

[['4fe12b19-280a-4286-a84a-979e04263a29',
  'f3bc7a75-21eb-4685-9da4-2626cd0b811c',
  '449e2c36-fe0e-49dd-8df8-85e8c176b90f'],
 ['4b030629-e531-4302-90ca-059b452b8807',
  '990c248e-ce91-49f0-9312-ee1fb04f6bc3',
  '440866c5-b3a6-4ce8-990e-ae991bd3406e'],
 ['8931e87d-dcba-49e9-b62f-0c293a479271',
  'b321dd63-ad57-48c6-974c-9b928032a3b5',
  'ab8b5d8b-84cd-4818-b897-517dd985a5dd']]

In [420]:
sessionsdf[sessionsdf["id"]=="440866c5-b3a6-4ce8-990e-ae991bd3406e"]

,id,range,order,event_id,title,session_type,location,start_time,end_time


In [421]:
event_id = sessionsdf.iloc[0]["event_id"]

In [422]:
sessionsdf["id"].unique()

array(['8e9849b4-69ba-45b1-bb80-3f6cba5503a2',
       'e995dd2b-68a4-409c-9139-35741e64296b',
       'ca8ab007-e0a3-48fe-b774-bef524571aa9',
       'bacee142-0f47-4d12-b9c1-d87756d0d0ce',
       'e0318b1e-1be7-4ba4-8fbd-229928c3eaaa',
       '82fd33f8-d528-4699-8bf3-8a8ad24b5e46',
       'ece7729b-77ac-43a5-abd3-d34e4b1cb6cc',
       '7c1c6cb3-d2ba-4938-8ed2-914f7d503245',
       'ecc14d1d-5d40-4dbb-9d1d-0120aa493e92',
       '327bfcbe-df29-4a43-82aa-cbec96f7019c',
       '62465bc8-3733-40a6-bc49-d8a30ecc8c39',
       '5422da19-8361-46ff-91f9-60a9862e5f30',
       '8a5557cb-1f7e-46b2-a568-759fe83cb26d',
       'e165ef71-0437-49aa-9efb-bd753117a97a',
       'e361c5e9-b20d-4b3e-870f-f297438092fd',
       '458d8bd1-a7c7-4572-84e5-c14023af247f',
       'a47c7549-e5d8-4a2b-9415-5c1b14f07fd4',
       '576f4ced-fbd5-4ab1-b5c6-3cf35becca08',
       'ea8d1987-d612-4398-b84c-fc7d707dc893',
       'b90481d0-96c4-49c8-8674-a4b61c057f5b',
       '91811043-0728-4318-9926-04d8762d9aaa',
       'fcdc7

In [423]:
# add an eposter session
sessionsdf = sessionsdf.append(pd.DataFrame([{
    "id": str(uuid.uuid4()),
    "range": 10000, "order": 10000, "event_id": event_id, "title": "e-posters session",
    "session_type": "e-posters session"}]))

In [424]:
# Reorder sessions
sessionsdf["rangeorder"] = sessionsdf["range"]*100 + sessionsdf["order"]
sessionsdf.sort_values(by="rangeorder", inplace=True)
sessionsdf.reset_index(inplace=True, drop=True)
sessionsdf["order"] = sessionsdf.index + 1

In [425]:
sessionsdf.drop(columns=["range"], inplace=True)

In [426]:
sessionsdf.head()

,id,order,event_id,title,session_type,location,start_time,end_time,rangeorder
0,23786821-0496-4a97-9d82-053ec08ea8e7,1,95271c09-7598-40e8-91b5-61712e726e86,ESMO Virtual Congress 2020 Opening ceremony,Channel 1,Channel 1,2020-09-19 11:15:00,2020-09-19 12:15:00,103
1,fbc4e213-3cb3-4f1d-b71c-32808f28d8c1,2,95271c09-7598-40e8-91b5-61712e726e86,EONS13: Opening session,Channel 4,Channel 4,2020-09-19 12:30:00,2020-09-19 14:00:00,104
2,f0631b5a-4726-458c-bf74-3228043809da,3,95271c09-7598-40e8-91b5-61712e726e86,EONS13: Quality and safety of cancer nursing,Channel 4,Channel 4,2020-09-21 16:20:00,2020-09-21 17:50:00,105
3,a21facb4-7d8c-486b-8eae-1abe56991cfa,4,95271c09-7598-40e8-91b5-61712e726e86,EONS13: Immunotherapy and precision oncology: ...,Channel 4,Channel 4,2020-09-20 12:30:00,2020-09-20 14:00:00,106
4,060df60f-564f-46c6-acee-281399cb1049,5,95271c09-7598-40e8-91b5-61712e726e86,EONS13: Age specific care: Adolescents and you...,Channel 4,Channel 4,2020-09-21 12:30:00,2020-09-21 14:00:00,107


In [427]:
with jsonlines.open('../output/structured/Session.jsonl', mode='w') as writer:
    sessionsdf["start_time"] = sessionsdf["start_time"].dt.strftime("%Y-%m-%dT%H:%M:%S.%f")
    sessionsdf["end_time"] = sessionsdf["end_time"].dt.strftime("%Y-%m-%dT%H:%M:%S.%f")
    writer.write_all(sessionsdf.T.apply(lambda x: x.dropna().to_dict()).tolist())

## All presentations, including e-posters

In [428]:
# Filter presentations
all_sessions_raw[all_sessions_raw["class"]=="presentation"].iloc[250]

id                            f6cd7724-0c67-43fe-b3e7-2d7f40380f5d
class                                                 presentation
range                                                          NaN
order                                                          0.0
event_id                      95271c09-7598-40e8-91b5-61712e726e86
title            How molecular biology influences radical surgi...
session_type                                                   NaN
content_id                                                     NaN
role                                                           NaN
full_name                                                      NaN
first_name                                                     NaN
last_name                                                      NaN
second_name                                                    NaN
country                                                        NaN
city                                                          

In [429]:
ids_mapping

[['4fe12b19-280a-4286-a84a-979e04263a29',
  'f3bc7a75-21eb-4685-9da4-2626cd0b811c',
  '449e2c36-fe0e-49dd-8df8-85e8c176b90f'],
 ['4b030629-e531-4302-90ca-059b452b8807',
  '990c248e-ce91-49f0-9312-ee1fb04f6bc3',
  '440866c5-b3a6-4ce8-990e-ae991bd3406e'],
 ['8931e87d-dcba-49e9-b62f-0c293a479271',
  'b321dd63-ad57-48c6-974c-9b928032a3b5',
  'ab8b5d8b-84cd-4818-b897-517dd985a5dd']]

In [430]:
def map_session_id(session_id):
    for mapping in ids_mapping:
        if session_id in mapping:
            return mapping[0]
    return session_id

In [431]:
for mapping in ids_mapping:
    print(mapping)

['4fe12b19-280a-4286-a84a-979e04263a29', 'f3bc7a75-21eb-4685-9da4-2626cd0b811c', '449e2c36-fe0e-49dd-8df8-85e8c176b90f']
['4b030629-e531-4302-90ca-059b452b8807', '990c248e-ce91-49f0-9312-ee1fb04f6bc3', '440866c5-b3a6-4ce8-990e-ae991bd3406e']
['8931e87d-dcba-49e9-b62f-0c293a479271', 'b321dd63-ad57-48c6-974c-9b928032a3b5', 'ab8b5d8b-84cd-4818-b897-517dd985a5dd']


In [432]:
map_session_id('b321dd63-ad57-48c6-974c-9b928032a3b5')

'8931e87d-dcba-49e9-b62f-0c293a479271'

In [433]:
all_sessions_raw["session_id"].unique()

array([nan, '8e9849b4-69ba-45b1-bb80-3f6cba5503a2',
       'e995dd2b-68a4-409c-9139-35741e64296b',
       'ca8ab007-e0a3-48fe-b774-bef524571aa9',
       'bacee142-0f47-4d12-b9c1-d87756d0d0ce',
       'e0318b1e-1be7-4ba4-8fbd-229928c3eaaa',
       '82fd33f8-d528-4699-8bf3-8a8ad24b5e46',
       'ece7729b-77ac-43a5-abd3-d34e4b1cb6cc',
       '7c1c6cb3-d2ba-4938-8ed2-914f7d503245',
       'ecc14d1d-5d40-4dbb-9d1d-0120aa493e92',
       '327bfcbe-df29-4a43-82aa-cbec96f7019c',
       '62465bc8-3733-40a6-bc49-d8a30ecc8c39',
       '5422da19-8361-46ff-91f9-60a9862e5f30',
       '8a5557cb-1f7e-46b2-a568-759fe83cb26d',
       'e165ef71-0437-49aa-9efb-bd753117a97a',
       'e361c5e9-b20d-4b3e-870f-f297438092fd',
       '458d8bd1-a7c7-4572-84e5-c14023af247f',
       'a47c7549-e5d8-4a2b-9415-5c1b14f07fd4',
       '576f4ced-fbd5-4ab1-b5c6-3cf35becca08',
       'ea8d1987-d612-4398-b84c-fc7d707dc893',
       'b90481d0-96c4-49c8-8674-a4b61c057f5b',
       '91811043-0728-4318-9926-04d8762d9aaa',
       '

In [434]:
all_sessions_raw["session_id"] = all_sessions_raw["session_id"].apply(
    map_session_id)

In [435]:
ids_mapping

[['4fe12b19-280a-4286-a84a-979e04263a29',
  'f3bc7a75-21eb-4685-9da4-2626cd0b811c',
  '449e2c36-fe0e-49dd-8df8-85e8c176b90f'],
 ['4b030629-e531-4302-90ca-059b452b8807',
  '990c248e-ce91-49f0-9312-ee1fb04f6bc3',
  '440866c5-b3a6-4ce8-990e-ae991bd3406e'],
 ['8931e87d-dcba-49e9-b62f-0c293a479271',
  'b321dd63-ad57-48c6-974c-9b928032a3b5',
  'ab8b5d8b-84cd-4818-b897-517dd985a5dd']]

In [436]:
all_sessions_raw[
    all_sessions_raw["session_id"]=="b321dd63-ad57-48c6-974c-9b928032a3b5"]

,id,class,range,order,event_id,title,session_type,content_id,role,full_name,...,second_name,country,city,session_order,session_range,session_id,location,start_time,end_time,description


In [437]:
presentationsdf = all_sessions_raw[all_sessions_raw["class"] == "presentation"][[
        "id", "title", "order", "start_time", "end_time", "session_id", "description",
        "session_order", "session_range"]]

In [ ]:
presentationsdf["session_id"].unique()

In [441]:
[item for item in presentationsdf.session_id if item not in list(sessionsdf.id)]

[]

In [166]:
#presentationsdf["session_id"] = presentationsdf["session_id"].apply(
#    map_session_id)

In [349]:
presentationsdf["session_id"].unique()

array(['8e9849b4-69ba-45b1-bb80-3f6cba5503a2',
       'e995dd2b-68a4-409c-9139-35741e64296b',
       'ca8ab007-e0a3-48fe-b774-bef524571aa9',
       'bacee142-0f47-4d12-b9c1-d87756d0d0ce',
       'e0318b1e-1be7-4ba4-8fbd-229928c3eaaa',
       '82fd33f8-d528-4699-8bf3-8a8ad24b5e46',
       'ece7729b-77ac-43a5-abd3-d34e4b1cb6cc',
       '7c1c6cb3-d2ba-4938-8ed2-914f7d503245',
       'ecc14d1d-5d40-4dbb-9d1d-0120aa493e92',
       '327bfcbe-df29-4a43-82aa-cbec96f7019c',
       '62465bc8-3733-40a6-bc49-d8a30ecc8c39',
       '5422da19-8361-46ff-91f9-60a9862e5f30',
       '8a5557cb-1f7e-46b2-a568-759fe83cb26d',
       'e165ef71-0437-49aa-9efb-bd753117a97a',
       'e361c5e9-b20d-4b3e-870f-f297438092fd',
       '458d8bd1-a7c7-4572-84e5-c14023af247f',
       'a47c7549-e5d8-4a2b-9415-5c1b14f07fd4',
       '576f4ced-fbd5-4ab1-b5c6-3cf35becca08',
       'ea8d1987-d612-4398-b84c-fc7d707dc893',
       'b90481d0-96c4-49c8-8674-a4b61c057f5b',
       '91811043-0728-4318-9926-04d8762d9aaa',
       'fcdc7

In [442]:
all_eposters_raw[all_eposters_raw["class"]=="eposter"].iloc[2]

id                          ae9f52e5-7a21-49da-8b98-87c67b0302ee
class                                                    eposter
range                                                        1.0
order                                                        2.0
event_id                    492e20bd-e514-43f6-9acc-517fda546d8b
title          11P - Preclinical evaluation of DS-2087b, a no...
start_time                                   2020-09-17 00:00:00
end_time                                     2020-09-17 00:00:00
description    Background In-frame insertions in exon 20 of t...
content_id                                                   NaN
role                                                         NaN
full_name                                                    NaN
first_name                                                   NaN
last_name                                                    NaN
second_name                                                  NaN
city                     

In [443]:
presentationspostersdf = all_eposters_raw[all_eposters_raw["class"] == "eposter"][[
        "id", "title", "range", "order", "start_time", "end_time", "description"
    ]]

In [444]:
presentationspostersdf["session_id"] = sessionsdf.iloc[-1]["id"]

In [445]:
presentationspostersdf["session_range"] = 100000
presentationspostersdf["session_order"] = 100000

In [446]:
presentationsdf = pd.concat([presentationsdf, presentationspostersdf])

In [447]:
presentationsdf.head()

,id,title,order,start_time,end_time,session_id,description,session_order,session_range,range
2,b650b8d2-a8cc-49ab-b6dd-9aa2d8dde527,Welcome and introduction,0.0,NaT,NaT,8e9849b4-69ba-45b1-bb80-3f6cba5503a2,NaN,3.0,8.0,NaN
4,28d8287b-0174-49c7-b167-09969948c500,Latest updates in molecular testing in NSCLC,1.0,NaT,NaT,8e9849b4-69ba-45b1-bb80-3f6cba5503a2,NaN,3.0,8.0,NaN
6,abbea22e-a336-454c-bdce-73c0a597c6e7,EGFR+ NSCLC: A personalised sequencing approach,2.0,NaT,NaT,8e9849b4-69ba-45b1-bb80-3f6cba5503a2,NaN,3.0,8.0,NaN
8,0c3f7f31-6913-43c4-8ad2-1e169acc84c3,ALK+ and ROS1+ NSCLC: Optimising patient outco...,3.0,NaT,NaT,8e9849b4-69ba-45b1-bb80-3f6cba5503a2,NaN,3.0,8.0,NaN
10,4e1169eb-636d-44a4-8a06-9c2e49f96d13,Live Q&A - 22 September 2020 - 15:30-16:00,4.0,NaT,NaT,8e9849b4-69ba-45b1-bb80-3f6cba5503a2,NaN,3.0,8.0,NaN


In [448]:
presentationsdf["order"] = presentationsdf["order"].apply(lambda x: int(x))
presentationsdf["session_order"] = presentationsdf["session_order"].apply(lambda x: int(x))
presentationsdf["session_range"] = presentationsdf["session_range"].apply(lambda x: int(x))

In [449]:
presentationsdf

,id,title,order,start_time,end_time,session_id,description,session_order,session_range,range
2,b650b8d2-a8cc-49ab-b6dd-9aa2d8dde527,Welcome and introduction,0,NaT,NaT,8e9849b4-69ba-45b1-bb80-3f6cba5503a2,NaN,3,8,NaN
4,28d8287b-0174-49c7-b167-09969948c500,Latest updates in molecular testing in NSCLC,1,NaT,NaT,8e9849b4-69ba-45b1-bb80-3f6cba5503a2,NaN,3,8,NaN
6,abbea22e-a336-454c-bdce-73c0a597c6e7,EGFR+ NSCLC: A personalised sequencing approach,2,NaT,NaT,8e9849b4-69ba-45b1-bb80-3f6cba5503a2,NaN,3,8,NaN
8,0c3f7f31-6913-43c4-8ad2-1e169acc84c3,ALK+ and ROS1+ NSCLC: Optimising patient outco...,3,NaT,NaT,8e9849b4-69ba-45b1-bb80-3f6cba5503a2,NaN,3,8,NaN
10,4e1169eb-636d-44a4-8a06-9c2e49f96d13,Live Q&A - 22 September 2020 - 15:30-16:00,4,NaT,NaT,8e9849b4-69ba-45b1-bb80-3f6cba5503a2,NaN,3,8,NaN
...,...,...,...,...,...,...,...,...,...,...
8471,f8562dc0-ddb6-48f1-a604-fab0adb01ac3,CN16 - Coping strategies and functional status...,7,2020-09-17,2020-09-17,35724013-f88c-4a5c-869f-0cb5b397164e,Background Functional status among patients wi...,100000,100000,152.0
8473,e15f71b5-4bb5-497e-b7ac-b321103468d7,CN17 - Predictors of distinct prognostic-aware...,8,2020-09-17,2020-09-17,35724013-f88c-4a5c-869f-0cb5b397164e,Background Accurate prognostic awareness (PA) ...,100000,100000,152.0
8475,eef00e5b-53b3-4af6-b9ed-ccaa4e39cbf9,CN18 - Promoting acceptance and execution of a...,9,2020-09-17,2020-09-17,35724013-f88c-4a5c-869f-0cb5b397164e,Background Advanced care planning (ACP) facili...,100000,100000,152.0
8477,577fe16b-dbad-428c-96c9-0e4acc8bc1b4,CN19 - Healthcare professionals' perception ab...,10,2020-09-17,2020-09-17,35724013-f88c-4a5c-869f-0cb5b397164e,Background End-of-life (EOL) conversations are...,100000,100000,152.0


In [450]:
presentationsdf["rangeorder"] = presentationsdf[
    "session_range"]*10000 + presentationsdf["session_order"]*100 + presentationsdf["order"]
presentationsdf["rangeorder"] = presentationsdf["rangeorder"].apply(int)
presentationsdf.drop(columns=["session_range", "session_order", "order"], inplace=True)

In [451]:
presentationsdf.sort_values(by="rangeorder", inplace=True)

In [452]:
presentationsdf.reset_index(inplace=True, drop=True)
presentationsdf["order"] = presentationsdf.index + 1
presentationsdf.drop(columns=["rangeorder"], inplace=True)

In [453]:
presentationsdf["description"] = presentationsdf["description"].apply(
    lambda x: np.nan if pd.isnull(x) else x)

In [454]:
presentationsdf.drop(columns=['range'], inplace=True)

In [455]:
presentationsdf.head()

,id,title,start_time,end_time,session_id,description,order
0,3e650d95-904f-4326-b7c7-3c326fbeec15,Introduction,2020-09-22 13:00:00,2020-09-22 13:05:00,8931e87d-dcba-49e9-b62f-0c293a479271,NaN,1
1,709a064b-3704-4ce2-a5f5-cdf41975873f,Introduction to RET mutations and fusions and ...,2020-09-22 13:05:00,2020-09-22 13:20:00,8931e87d-dcba-49e9-b62f-0c293a479271,NaN,2
2,7f845162-c324-4f5e-9420-19d44937e71f,RET fusion TKI: Present results and recent dev...,2020-09-22 13:20:00,2020-09-22 13:35:00,8931e87d-dcba-49e9-b62f-0c293a479271,NaN,3
3,99183f93-126e-4aad-bd9f-c9db00add8e2,RET alterations in Thyroid carcinomas: Results...,2020-09-22 13:35:00,2020-09-22 13:50:00,8931e87d-dcba-49e9-b62f-0c293a479271,NaN,4
4,5c41a2b8-3ce2-44cf-b6a5-c91eea70a9e1,RET determination: Present recommendation of t...,2020-09-22 13:50:00,2020-09-22 14:05:00,8931e87d-dcba-49e9-b62f-0c293a479271,NaN,5


In [456]:
sessionsdf["event_id"].unique()

array(['95271c09-7598-40e8-91b5-61712e726e86'], dtype=object)

In [457]:
[item for item in presentationsdf.session_id if item not in list(sessionsdf.id)]

[]

In [458]:
with jsonlines.open('../output/structured/Presentation.jsonl', mode='w') as writer:
    presentationsdf["start_time"] = presentationsdf["start_time"].dt.strftime("%Y-%m-%dT%H:%M:%S.%f")
    presentationsdf["end_time"] = presentationsdf["end_time"].dt.strftime("%Y-%m-%dT%H:%M:%S.%f")
    writer.write_all(presentationsdf.T.apply(lambda x: x.dropna().to_dict()).tolist())

## Persons

In [459]:
all_eposters_raw["class"].unique()

array(['eposter', 'posterperson'], dtype=object)

In [460]:
all_sessions_raw["class"].unique()

array(['session', 'sessionperson', 'presentation', 'presentationperson'],
      dtype=object)

In [461]:
personsdf = all_sessions_raw[all_sessions_raw["class"].isin(['sessionperson', 'presentationperson'])][[
    "id", "event_id", "full_name", "first_name", "last_name", "second_name", "country", "city", "title"
]]

In [463]:
eposterspersondf = all_eposters_raw[all_eposters_raw["class"].isin(['sessionperson', 'presentationperson'])][[
    "id", "event_id", "full_name", "first_name", "last_name", "second_name", "country", "city", "title"
]]

In [464]:
personsdf = pd.concat([personsdf, eposterspersondf])

In [465]:
len(personsdf)

3806

In [466]:
personsdf = personsdf.drop_duplicates(subset="full_name", keep='first')

In [467]:
len(personsdf)

2436

In [468]:
personsdf["id"] = str(uuid.uuid4())

In [469]:
personsdf.head()

,id,event_id,full_name,first_name,last_name,second_name,country,city,title
1,65f2e3e3-8c1c-4bce-aaf4-8df8b66402cd,95271c09-7598-40e8-91b5-61712e726e86,Rosario Garcia Campelo,Rosario,Campelo,Garcia,Spain,A Coruña,
5,65f2e3e3-8c1c-4bce-aaf4-8df8b66402cd,95271c09-7598-40e8-91b5-61712e726e86,Justin F. Gainor,Justin,Gainor,F.,A,Boston,
7,65f2e3e3-8c1c-4bce-aaf4-8df8b66402cd,95271c09-7598-40e8-91b5-61712e726e86,Jesus Corral Jaime,Jesus,Jaime,Corral,Spain,Pamplona,
9,65f2e3e3-8c1c-4bce-aaf4-8df8b66402cd,95271c09-7598-40e8-91b5-61712e726e86,Todd Bauer,Todd,Bauer,,United States of America,Nashville,
12,65f2e3e3-8c1c-4bce-aaf4-8df8b66402cd,95271c09-7598-40e8-91b5-61712e726e86,Amanda Drury,Amanda,Drury,,Ireland,Dublin,


In [470]:
personsdf["event_id"].unique()[0]

'95271c09-7598-40e8-91b5-61712e726e86'

In [471]:
with jsonlines.open('../output/structured/Person.jsonl', mode='w') as writer:
    writer.write_all(personsdf.T.apply(lambda x: x.dropna().to_dict()).tolist())

## SessionPerson and PresentationPerson

In [472]:
sessionpersonsdf = all_sessions_raw[all_sessions_raw["class"]=="sessionperson"][[
    "full_name", "role", "content_id"]]

In [474]:
[item for item in sessionpersonsdf.content_id if item not in list(sessionsdf.id)]

['f3bc7a75-21eb-4685-9da4-2626cd0b811c',
 'f3bc7a75-21eb-4685-9da4-2626cd0b811c',
 '990c248e-ce91-49f0-9312-ee1fb04f6bc3',
 '990c248e-ce91-49f0-9312-ee1fb04f6bc3',
 'b321dd63-ad57-48c6-974c-9b928032a3b5',
 'b321dd63-ad57-48c6-974c-9b928032a3b5',
 '440866c5-b3a6-4ce8-990e-ae991bd3406e',
 '440866c5-b3a6-4ce8-990e-ae991bd3406e',
 'ab8b5d8b-84cd-4818-b897-517dd985a5dd',
 'ab8b5d8b-84cd-4818-b897-517dd985a5dd',
 '449e2c36-fe0e-49dd-8df8-85e8c176b90f',
 '449e2c36-fe0e-49dd-8df8-85e8c176b90f']

In [475]:
sessionpersonsdf["content_id"] = sessionpersonsdf["content_id"].apply(
    map_session_id)

In [476]:
[item for item in sessionpersonsdf.content_id if item not in list(sessionsdf.id)]

[]

In [477]:
sessionpersonsdf.rename(columns={"content_id": "session_id"}, inplace=True)

In [478]:
#all_sessions_raw["session_id"] = all_sessions_raw["session_id"].apply(
#    lambda x: first_duplicated_sessions_id if x in list_duplicated_sessions_id else x)

In [479]:
sessionpersonsdf = sessionpersonsdf.merge(personsdf[["full_name", "id"]], on="full_name").rename(
    columns={"id": "person_id"})

In [480]:
sessionpersonsdf.drop(columns=["full_name"], inplace=True)

In [481]:
sessionpersonsdf.head()

,role,session_id,person_id
0,chairperson,8e9849b4-69ba-45b1-bb80-3f6cba5503a2,65f2e3e3-8c1c-4bce-aaf4-8df8b66402cd
1,chairperson,e995dd2b-68a4-409c-9139-35741e64296b,65f2e3e3-8c1c-4bce-aaf4-8df8b66402cd
2,chairperson,ca8ab007-e0a3-48fe-b774-bef524571aa9,65f2e3e3-8c1c-4bce-aaf4-8df8b66402cd
3,chairperson,3cf8cf38-6924-4a6c-96af-4611245149bf,65f2e3e3-8c1c-4bce-aaf4-8df8b66402cd
4,chairperson,bacee142-0f47-4d12-b9c1-d87756d0d0ce,65f2e3e3-8c1c-4bce-aaf4-8df8b66402cd


In [482]:
with jsonlines.open('../output/structured/SessionPerson.jsonl', mode='w') as writer:
    writer.write_all(sessionpersonsdf.T.apply(lambda x: x.dropna().to_dict()).tolist())

In [483]:
presentationpersonsdf = all_sessions_raw[all_sessions_raw["class"]=="presentationperson"][[
    "full_name", "role", "content_id"]]

In [484]:
presentationpersonsdf.head()

,full_name,role,content_id
3,Rosario Garcia Campelo,presenter,b650b8d2-a8cc-49ab-b6dd-9aa2d8dde527
5,Justin F. Gainor,presenter,28d8287b-0174-49c7-b167-09969948c500
7,Jesus Corral Jaime,presenter,abbea22e-a336-454c-bdce-73c0a597c6e7
9,Todd Bauer,presenter,0c3f7f31-6913-43c4-8ad2-1e169acc84c3
14,Johan De Munter,presenter,1caeff56-65d7-4635-ae1a-f0f7d0399c1b


In [485]:
[item for item in presentationpersonsdf.content_id if item not in list(presentationsdf.id)]

[]

In [486]:
all_eposters_raw['class'].unique()

array(['eposter', 'posterperson'], dtype=object)

In [487]:
eposterspresentationpersonsdf = all_eposters_raw[all_eposters_raw["class"]=="posterperson"][[
    "full_name", "role", "content_id"]]

In [488]:
eposterspresentationpersonsdf.head()

,full_name,role,content_id
2,Yanwei Shen,speaker,4b63d427-6852-4cca-9096-ae03b367817b
4,Yasuhito Nagamoto,speaker,ae9f52e5-7a21-49da-8b98-87c67b0302ee
5,M. Miyamoto,author,ae9f52e5-7a21-49da-8b98-87c67b0302ee
6,N. Togashi,author,ae9f52e5-7a21-49da-8b98-87c67b0302ee
7,T. Taira,author,ae9f52e5-7a21-49da-8b98-87c67b0302ee


In [489]:
presentationpersonsdf = pd.concat([presentationpersonsdf, eposterspresentationpersonsdf])

In [490]:
presentationpersonsdf.rename(columns={"content_id": "presentation_id"}, inplace=True)

In [491]:
presentationpersonsdf.head()

,full_name,role,presentation_id
3,Rosario Garcia Campelo,presenter,b650b8d2-a8cc-49ab-b6dd-9aa2d8dde527
5,Justin F. Gainor,presenter,28d8287b-0174-49c7-b167-09969948c500
7,Jesus Corral Jaime,presenter,abbea22e-a336-454c-bdce-73c0a597c6e7
9,Todd Bauer,presenter,0c3f7f31-6913-43c4-8ad2-1e169acc84c3
14,Johan De Munter,presenter,1caeff56-65d7-4635-ae1a-f0f7d0399c1b


In [492]:
presentationpersonsdf["role"].unique()

array(['presenter', 'author', 'speaker'], dtype=object)

In [493]:
presentationpersonsdf[pd.isnull(presentationpersonsdf["presentation_id"])]

,full_name,role,presentation_id


In [494]:
presentationpersonsdf = presentationpersonsdf.merge(personsdf[["full_name", "id"]], on="full_name").rename(
    columns={"id": "person_id"})

In [495]:
presentationpersonsdf.drop(columns=["full_name"], inplace=True)

In [496]:
presentationpersonsdf.head()

,role,presentation_id,person_id
0,presenter,b650b8d2-a8cc-49ab-b6dd-9aa2d8dde527,65f2e3e3-8c1c-4bce-aaf4-8df8b66402cd
1,presenter,a46ed4ab-8288-4bfe-9bbf-d70e1db264ef,65f2e3e3-8c1c-4bce-aaf4-8df8b66402cd
2,presenter,28d8287b-0174-49c7-b167-09969948c500,65f2e3e3-8c1c-4bce-aaf4-8df8b66402cd
3,presenter,abbea22e-a336-454c-bdce-73c0a597c6e7,65f2e3e3-8c1c-4bce-aaf4-8df8b66402cd
4,presenter,0c3f7f31-6913-43c4-8ad2-1e169acc84c3,65f2e3e3-8c1c-4bce-aaf4-8df8b66402cd


In [497]:
with jsonlines.open('../output/structured/PresentationPerson.jsonl', mode='w') as writer:
    writer.write_all(presentationpersonsdf.T.apply(lambda x: x.dropna().to_dict()).tolist())

In [498]:
eventdf = pd.DataFrame([{
    "airfinity_id": "airfinity_id", "city": "virtual", "country": "virtual",
    "downloaded_at": "2021-05-17T15:32:08.464063", "end_date": "2020-10-18",
    "id": personsdf["event_id"].unique()[0], "name": "ESMO virtual congress 2020",
    "source_platform": "ctimeetingtech",
    "source_url": "https://cslide.ctimeetingtech.com/esmo2020/attendee/confcal/session/calendar/2020-09-19",
    "start_date": "2020-09-15"
}])

In [499]:
with jsonlines.open('../output/structured/Event.jsonl', mode='w') as writer:
    writer.write_all(eventdf.T.apply(lambda x: x.dropna().to_dict()).tolist())